In [ ]:
import torch
import zoib

In [ ]:
zoib_fit = zoib.ZOIBeta(p=torch.tensor([.5]), 
               q=torch.tensor([.3]), 
               concentration1=torch.tensor([1.]), 
               concentration0=torch.tensor([1.]))

In [ ]:
zoib_fit.log_prob(torch.tensor([0.])) 

In [ ]:
zoib_fit.log_prob(torch.tensor([1.]))

In [ ]:
zoib_fit.log_prob(torch.tensor([0.4]))

In [ ]:
zoib_fit.log_prob(torch.tensor([.9]))

In [ ]:
zoib_fit.log_prob(torch.tensor([.2]))

In [ ]:
# Test full loss
pred = torch.tensor([[0.5, 0.3, 1., 1.],[0.5, 0.3, 1., 1.], [0.5, 0.3, 1., 1.], [0.5, 0.3, 1., 1.], [0.5, 0.3, 1., 1.]])
true_y = torch.tensor([0., 1., 0.4, 0.9, 0.2])
log_probs = zoib.zoib_loss(
    pred,
    true_y,# Flatten to (batch_size*timesteps) X 1
    return_mean=False) 
print(log_probs)# [0.6931, 1.8971, 1.0498, 1.0499, 1.0498]

mean_loss = zoib.zoib_loss(
    pred,
    true_y,# Flatten to (batch_size*timesteps) X 1
    return_mean=True)
print(mean_loss) # 1.1479
